# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
%pip install azureml-train-automl-runtime==1.57.0 -q
%pip install azureml-widgets -q
%pip install --upgrade azureml-sdk[notebooks,automl] -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-ml 0.6.1 requires enum34, which is not installed.
scikit-image 0.24.0 requires networkx>=2.8, but you have networkx 2.5 which is incompatible.
dask-sql 2024.5.0 requires dask[dataframe]>=2024.4.1, but you have dask 2023.2.0 which is incompatible.
dask-sql 2024.5.0 requires distributed>=2024.4.1, but you have distributed 2023.2.0 which is incompatible.
dask-expr 1.1.14 requires dask==2024.9.0, but you have dask 2023.2.0 which is incompatible.
dask-expr 1.1.14 requires pandas>=2, but you have pandas 1.5.3 which is incompatible.
azureml-train 1.57.0 requires azureml-train-core~=1.57.0, but you have azureml-train-core 1.58.0 which is incompatible.
azureml-tensorboard 1.57.0 require

In [2]:
import os
import csv
import logging
import json
import requests 

import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources
from matplotlib import pyplot as plt

import azureml.core
from azureml.train.automl import AutoMLConfig
from azureml.core.experiment import Experiment
from azureml.core.dataset import Dataset
from azureml.pipeline.steps import AutoMLStep
from azureml.core.workspace import Workspace
from azureml.widgets import RunDetails

from azureml.core.compute import AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.compute import ComputeTarget



## Create Workspace

In [3]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')


Workspace name: quick-starts-ws-269873
Azure region: westus2
Subscription id: 3e42d11f-d64d-4173-af9b-12ecaa1030b3
Resource group: aml-quickstarts-269873


## Create Experiment

In [4]:
# choose a name for experiment
experiment_name = 'azmlproject3'
project_folder = './capstone-project'

experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
azmlproject3,quick-starts-ws-269873,Link to Azure Machine Learning studio,Link to Documentation


## Create compute cluster

In [5]:
# name compute cluster
amlcompute_cluster_name = "azmlproject3"

# check use compute cluster if exist, else create
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Not found existed compute cluster, creating one...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D3_v2',
                                                           min_nodes=1,
                                                           max_nodes=72)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)
    print('Create new compute cluster successfully')

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


## Dataset

In [6]:
found = False
key = "credit-default"
description_text = "credit card default dataset"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        data_url = 'https://raw.githubusercontent.com/TungPhamDuy/azmlproject3/refs/heads/main/credit-card-default.csv'
        dataset = Dataset.Tabular.from_delimited_files(data_url)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.head(5)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,id,linitBal,sex,education,married,age,pay_0,pay_2,pay_3,pay_4,...,billAmt_4,billAmt_5,billAmt_6,payAmt_1,payAmt_2,payAmt_3,payAmt_4,payAmt_5,payAmt_6,defaulted
0,1,20000,False,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,True
1,2,120000,False,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,True
2,3,90000,False,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,False
3,4,50000,False,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,False
4,5,50000,True,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,False


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
automl_settings = {
    "experiment_timeout_hours": 0.5,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy',
    "iterations": 20
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="defaulted",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
# TODO: Submit your experiment
automl_run = experiment.submit(config=automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on azmlproject3 with default configuration
Running on remote compute: azmlproject3


Experiment,Id,Type,Status,Details Page,Docs Page
azmlproject3,AutoML_cd39e834-d22d-423a-84be-cad14b577e7b,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  In order to accurately evaluate the model(s) trained by AutoML, we leverage a dataset that the model is not trained on. Hence, if the user doesn't provide an explicit validation dataset, a part of the training dataset is used to achieve this. For smaller datasets (fewer than 20,000 samples), cross-validation is leveraged, else a single hold-out set is split from the training data to serve as the validation dataset. Hence, for your input data we leverage cross-validation with 10 folds, if the number o

In [10]:
# Show the RunDetails widget
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
azmlproject3,AutoML_cd39e834-d22d-423a-84be-cad14b577e7b,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  In order to accurately evaluate the model(s) trained by AutoML, we leverage a dataset that the model is not trained on. Hence, if the user doesn't provide an explicit validation dataset, a part of the training dataset is used to achieve this. For smaller datasets (fewer than 20,000 samples), cross-validation is leveraged, else a single hold-out set is split from the training data to serve as the validation dataset. Hence, for your input data we leverage cross-validation with 10 folds, if the number of training samples are fewer than 1000, and 3 folds in all other cases.
              Learn more about cross validation: https://aka.ms/AutomatedMLCrossValidation
DETAILS:      
+------------------------------+
|Number of folds               |
+==============================+
|3                             |
+------

{'runId': 'AutoML_cd39e834-d22d-423a-84be-cad14b577e7b',
 'target': 'azmlproject3',
 'status': 'Completed',
 'startTimeUtc': '2024-11-18T07:09:30.303019Z',
 'endTimeUtc': '2024-11-18T07:19:28.424593Z',
 'services': {},
 'properties': {'num_iterations': '20',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'azmlproject3',
  'AMLSettingsJsonString': '{"path":null,"name":"azmlproject3","subscription_id":"3e42d11f-d64d-4173-af9b-12ecaa1030b3","resource_group":"aml-quickstarts-269873","workspace_name":"quick-starts-ws-269873","region":"westus2","compute_target":"azmlproject3","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":true,"enable_parallel_run":false,"num_procs":null,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation"

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [11]:
# Retrieve and save your best automl model.
best, model = automl_run.get_output()
best

Experiment,Id,Type,Status,Details Page,Docs Page
azmlproject3,AutoML_cd39e834-d22d-423a-84be-cad14b577e7b_18,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [12]:
model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, is_cross_validation=True, working_dir='/mnt/batch/tasks/shared/LS_root/mounts/clusters/azmlproject3/code/Users/odl_user_269873')),
                                                 ('prefittedsoftvotingclassifier',
                                                  PreFittedSoftVotingClassifier(classification_...g_lambda=1.0416666666666667, subsample=1, tree_method='auto'))]))], flatten_transform=False, weights=[0.07142857142857142, 0.07142857142857142, 0.14285714285714285, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.35714285714285715]))],
                                       'verbose': False},
                             y_transformer={},
                             y_transformer_name='LabelEncoder')

In [13]:
registered_model = best.register_model(
    model_path="outputs/model.pkl",
    model_name="best_automl",
    properties={"accuracy": best.get_metrics()["accuracy"]}
)

In [14]:
best.get_metrics()

{'precision_score_weighted': 0.8956806277555409,
 'recall_score_macro': 0.8942066787286196,
 'f1_score_macro': 0.889435611315247,
 'AUC_micro': 0.9519727861178332,
 'recall_score_micro': 0.8897003885544867,
 'recall_score_weighted': 0.8897003885544867,
 'average_precision_score_weighted': 0.9316330412650721,
 'log_loss': 0.3935290186338242,
 'average_precision_score_macro': 0.9279934912929114,
 'AUC_macro': 0.9403625715157627,
 'average_precision_score_micro': 0.952954992243291,
 'AUC_weighted': 0.940362571515763,
 'balanced_accuracy': 0.8942066787286196,
 'norm_macro_recall': 0.7884133574572392,
 'accuracy': 0.8897003885544867,
 'precision_score_micro': 0.8897003885544867,
 'precision_score_macro': 0.8901182662061179,
 'weighted_accuracy': 0.8852979581873682,
 'f1_score_weighted': 0.889991033781205,
 'f1_score_micro': 0.8897003885544867,
 'matthews_correlation': 0.7843127863283691,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_cd39e834-d22d-423a-84be-cad14b577e7b_18/a

In [15]:
best.download_file('outputs/conda_env_v_1_0_0.yml','conda_env.yml')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [16]:
best.download_file('outputs/scoring_file_v_1_0_0.py', 'scoring.py')

TODO: In the cell below, send a request to the web service you deployed to test it.

In [17]:
from azureml.core import Model
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

env = best.get_environment()
infer_config = InferenceConfig(entry_script = 'scoring.py', environment = env)

# deloyment config
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                    memory_gb = 2, 
                                                    auth_enabled= True, 
                                                    enable_app_insights= True)
# model deloy
service = Model.deploy(ws, "deloy-best-model1", 
                       [registered_model], 
                       infer_config, 
                       deployment_config)
# wait for deloyment
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-11-18 07:22:15+00:00 Creating Container Registry if not exists.
2024-11-18 07:22:16+00:00 Use the existing image.
2024-11-18 07:22:18+00:00 Submitting deployment to compute.
2024-11-18 07:22:27+00:00 Checking the status of deployment deloy-best-model1..
2024-11-18 07:26:04+00:00 Checking the status of inference endpoint deloy-best-model1.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, print the logs of the web service and delete the service

In [25]:
# get 5 data from the df to test 
df_test = df.sample(5)
df_labeled = df_test.pop('defaulted')

# turn data to json format
send_request = json.dumps({'data': df_test.to_dict(orient='records')})
print(send_request)

{"data": [{"id": 7292, "linitBal": 30000, "sex": false, "education": 1, "married": 1, "age": 35, "pay_0": 2, "pay_2": 0, "pay_3": 0, "pay_4": 0, "pay_5": 0, "pay_6": 0, "billAmt_1": 27703, "billAmt_2": 28743, "billAmt_3": 27048, "billAmt_4": 26592, "billAmt_5": 23874, "billAmt_6": 23293, "payAmt_1": 1500, "payAmt_2": 1356, "payAmt_3": 900, "payAmt_4": 1000, "payAmt_5": 1100, "payAmt_6": 664}, {"id": 4168, "linitBal": 150000, "sex": true, "education": 3, "married": 1, "age": 34, "pay_0": 0, "pay_2": 0, "pay_3": 0, "pay_4": 0, "pay_5": 0, "pay_6": 0, "billAmt_1": 38808, "billAmt_2": 39879, "billAmt_3": 40589, "billAmt_4": 41551, "billAmt_5": 42420, "billAmt_6": 43105, "payAmt_1": 2000, "payAmt_2": 1674, "payAmt_3": 1636, "payAmt_4": 1690, "payAmt_5": 1544, "payAmt_6": 1751}, {"id": 1961, "linitBal": 30000, "sex": false, "education": 2, "married": 2, "age": 37, "pay_0": 3, "pay_2": 2, "pay_3": 2, "pay_4": 0, "pay_5": 0, "pay_6": 0, "billAmt_1": 29657, "billAmt_2": 30320, "billAmt_3": 2899

In [26]:
auth_key = service.get_keys()[0]
headers = {'Content-type': 'application/json'}
headers['Authorization'] = f'Bearer {auth_key}'
model_response = requests.post(service.scoring_uri, send_request, headers=headers)

In [27]:
print('Expected actual output:', list(df_labeled))
print('Response from model:', model_response.text)

Expected actual output: [True, False, True, True, False]
Response from model: "{\"result\": [true, false, true, true, false]}"


In [21]:
service.get_logs()

'2024-11-18T07:25:54,946920621+00:00 - rsyslog/run \n2024-11-18T07:25:54,953092562+00:00 - gunicorn/run \n2024-11-18T07:25:54,956597241+00:00 | gunicorn/run | \n2024-11-18T07:25:54,957391172+00:00 | gunicorn/run | ###############################################\n2024-11-18T07:25:54,958236738+00:00 | gunicorn/run | AzureML Container Runtime Information\n2024-11-18T07:25:54,959031019+00:00 | gunicorn/run | ###############################################\n2024-11-18T07:25:54,959866096+00:00 | gunicorn/run | \n2024-11-18T07:25:54,968367729+00:00 | gunicorn/run | \n2024-11-18T07:25:54,968505900+00:00 - nginx/run \n2024-11-18T07:25:54,973718719+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20240918.v1\n2024-11-18T07:25:54,975233210+00:00 | gunicorn/run | \n2024-11-18T07:25:54,977406873+00:00 | gunicorn/run | \n2024-11-18T07:25:54,980161950+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/

In [22]:
#service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
